In [ ]:
import numpy as np
import os, json
import pandas as pd
import random
import sentencepiece
import tqdm
import transformers
import torch
from torch.utils.data import DataLoader

import re

from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv("/content/sample_data/main_df.csv")

In [ ]:
data.head()

,Unnamed: 0,tweet_id,text,class,dataset,group
0,0,32520780256382976,#SoutherN HospitalitY :),0.0,stratified,Not specific
1,1,69219602424532992,"Eastern conference finals, MIAMI HEAT vs CHICA...",0.0,stratified,Not specific
2,2,88730854100189184,RT @USER: RT @USER: You're an idiot RT @USER: ...,2.0,stratified,lgbtq+
3,3,97589827465256960,RT @USER: If Ʊ r a muslim and Ʊ won't fast Ʊ r...,1.0,stratified,Not specific
4,4,121963728618012672,After all d noise you go and ask dsame questio...,1.0,stratified,Not specific


In [ ]:
data["class"] = data["class"].astype(int)

In [ ]:
data["class"].value_counts()

,count
class,
0,31491
1,2229
2,2058


In [ ]:
downsample_class_0 = data[data["class"]==0][:2500]
df = data[data["class"]!=0]
df = pd.concat([downsample_class_0, df])
df.shape

(6787, 6)

In [ ]:
base_df = pd.read_csv("/content/sample_data/igbo_train.csv")
base_df["tweet_id"] = base_df["tweet_id"].astype(int)
base_df_tweet_ids = base_df["Unnamed: 0"].to_list()
len(base_df_tweet_ids)

4750

In [ ]:
base_df.head()

,Unnamed: 0,tweet_id,text,class,dataset,group,clean_text,igbo
0,5002,1404223038460248064,@USER @USER @USER The killings in zamfara and ...,2,eval,Multi-groups,The killings in zamfara and some northern p...,Ọgbụgba ndụ ndị a na Zamfara na akụkụ ụfọdụ nk...
1,1367,1389216337248067584,@USER C is not in the Yoruba Alphabet.,0,stratified,Not specific,C is not in the Yoruba Alphabet.,C adịghị na Yoruba Alphabet.
2,75,360106230263455744,"Parasites,Pretenders and interlopers should st...",0,stratified,Not specific,"Parasites,Pretenders and interlopers should st...","Ndị na-eme ihe ọjọọ, ndị na-achọ ihe ọjọọ na n..."
3,1654,420480440353693696,"...desperate measures... ""@USER: Lol ""@USER: S...",0,al,Not specific,"...desperate measures... "": Lol "": Someone K-e...","...ihe ndị na-enweghị isi... "": Lol "": Onye ọ ..."
4,6336,805125996886232960,Heisman contender to Heisman contender.\n\nMay...,0,random,Not specific,Heisman contender to Heisman contender.\n\nMay...,"Mayfield hụrụ Westbrook, onye na-agba okpu okp..."


In [ ]:
train = df[df["Unnamed: 0"].isin(base_df_tweet_ids)]
test = df[~df["Unnamed: 0"].isin(base_df_tweet_ids)]

In [ ]:
train.head()

,Unnamed: 0,tweet_id,text,class,dataset,group
0,0,32520780256382976,#SoutherN HospitalitY :),0,stratified,Not specific
5,5,162810437371559936,RT @USER: Hmmm...my oyinbo friend wants to kno...,0,stratified,Not specific
11,11,205538285714550784,"An Igbo lady, sat together wif †̥ό̲ men in Gol...",0,stratified,Not specific
13,13,209601887102701568,RT @USER: “@USER: @USER there is trouble betwe...,0,stratified,Not specific
26,26,233402513385218048,"Say: ""In the Bounty of Allah, and in His Mercy...",0,stratified,Not specific


In [ ]:
print(len(train), len(test))

4750 2037


In [ ]:
# non_hate_speech = data[data["class"]==0]
# neu_speech = data[data["class"]==1]
# hate_speech = data[data["class"]==2]

In [ ]:
import re
def clean_tweet(tweet):
#     # Remove URLs
#     tweet = re.sub(r'http\S+', '', tweet)

    # Remove mentions and hashtags
    tweet = re.sub(r'@[A-Za-z0-9_]+|#[A-Za-z0-9_]+', '', tweet)

#     # Remove special characters, numbers, and punctuation
#     tweet = re.sub(r'[^A-Za-z\s]', '', tweet)

    # Remove 'RT' (Retweet) indicator
    tweet = re.sub(r'\bRT\b', '', tweet)

#     # Convert to lowercase
#     tweet = tweet.lower()

    return tweet

train["clean_text"] = train["text"].apply(clean_tweet)
test["clean_text"] = test["text"].apply(clean_tweet)

<ipython-input-43-1d1ec3c6897a>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["clean_text"] = train["text"].apply(clean_tweet)
<ipython-input-43-1d1ec3c6897a>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["clean_text"] = test["text"].apply(clean_tweet)


In [ ]:
train.head()

,Unnamed: 0,tweet_id,text,class,dataset,group,clean_text
0,0,32520780256382976,#SoutherN HospitalitY :),0,stratified,Not specific,HospitalitY :)
5,5,162810437371559936,RT @USER: Hmmm...my oyinbo friend wants to kno...,0,stratified,Not specific,: Hmmm...my oyinbo friend wants to know if 'M...
11,11,205538285714550784,"An Igbo lady, sat together wif †̥ό̲ men in Gol...",0,stratified,Not specific,"An Igbo lady, sat together wif †̥ό̲ men in Gol..."
13,13,209601887102701568,RT @USER: “@USER: @USER there is trouble betwe...,0,stratified,Not specific,: “: there is trouble between hausas and the...
26,26,233402513385218048,"Say: ""In the Bounty of Allah, and in His Mercy...",0,stratified,Not specific,"Say: ""In the Bounty of Allah, and in His Mercy..."


In [ ]:
# afriberta = afriberta.rename(columns={"text": "ibo"})
# afriberta.head(2)

In [ ]:
# ## keep updating this batch index interval values. Use 2000 rows
# l = 26000
# h = 28000
# sample = afriberta[l:h]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from tqdm import tqdm

model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")

translator = pipeline('translation', model=model, tokenizer=tokenizer, src_lang='eng_Latn', tgt_lang="yor_Latn", device=0)

def translate_text(df, column_name):
    tqdm.pandas()  # Use tqdm with pandas
    trans_list = translator(df[column_name].to_list(), max_length = 600)
    df['yoruba'] = [i["translation_text"] for i in trans_list]
    return df

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Apply the translation function to the DataFrame
train = translate_text(train, 'clean_text')

In [ ]:
train.to_csv("yoruba_train.csv", index=False)